# Webscraping

## Developer tools

### Let op
Als je al bekend bent met DeveloperTools zou je dit evt mogen overslaan. Wat wel belangrijk is is het gebruik van 'Network Tab' en XHR. Als je daar alles al van weet, mag je dit overslaan. 

Naast Python gaan we ook werken met de [Developer tools] van de web browser.  
[Chrome] en [Firefox] hebben dat beiden in hun browser.  


De toets `F12` of de sneltoetsen `Shift` + `Control` + `I` opent de developer tools van de browser.      


[Developer tools]: https://en.wikipedia.org/wiki/Web_development_tools
[Chrome]: https://developer.chrome.com/docs/devtools/open/ "Chrome Developer tools"
[Firefox]: https://developer.mozilla.org/en-US/docs/Learn/Common_questions/What_are_browser_developer_tools "Firefox Developer tools"

## Oefening met developer tools

### Uitleg: Demo pagina DemoBlaze

- Open een browser navigeer naar https://www.demoblaze.com/ en de developer tools.
- Zet de opties Preserve logs, Disable cache aan en filter op XHR  

> ![Developer tools](./img/devtools_XHR_filter.png "Preserve logs, Disable cache, filter XHR")

Open de demo website, we zien dat er wat XHR type requests gedaan wordt.

De url https://www.demoblaze.com/config.json wordt opgevraagt met de GET method.  

> ![Developer tools XHR get](./img/devtools_XHR_get_request.png "GET request, XHR type response")





Observeer de gegevens die te zien zijn in de response data. 

Als we er op klikken zien we de data die verstuurd en terug gekregen is.  
Onder de tab Response krijgen we de _Raw response data_ te zien.  
Dit kan van alles zijn, HTML, javascript en in dit geval JSON.  

> ![Developer tools XHR response](./img/devtools_XHR_response.png "raw response")

- **Verder is er nog veel interessante data te zien in andere XHR requests, kijken of je die kunt vinden!**



### Oefening: Demo pagina Computer database

Open een browser en navigeer naar de demo pagina https://computer-database.gatling.io/computers 

Open de developer tools.  
Klik op de next knop op de demo pagina.
In de developer tools open kan er worden gezien dat er een GET request is geweest.  
De request en de URL in de browser hebben een query.  

> ![query string params](./img/devtools_query_params.png "Query String Parameters")

Met wat giswerk kan er worden bedacht wat de query bevat.

`p=0` de key `p` en de value als cijfer moet waarschijnlijk de pagina voorstellen.  
`n=10` de key `n` is hoogstwaarschijnlijk het aantal items in de [table] op de pagina.  
`s=name` de [table] is gesorteerd op naam, dus `s` is voor _sort_.  
`d=asc` moet dan voor de manier van sorteren zijn, `asc` staat voor _ascending_.

[table]: https://developer.mozilla.org/en-US/docs/Learn/HTML/Tables/Basics


Met deze kennis kunnen we een query maken voor een tabel met de benodigde regels.  
De URL + query kan dan met Python-requests worden opgevraagd.

In [8]:
from urllib.parse import urljoin

import requests
from bs4 import BeautifulSoup

In [9]:
base_url = 'https://computer-database.gatling.io/computers'
query = '?p={page}&n={rows}'
url = urljoin(base_url, query.format(page=0, rows=5))  # pagina 0, regels 5
url

'https://computer-database.gatling.io/computers?p=0&n=5'

De URL is gecreeerd en

In [10]:
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')

Het is niet nodig maar notebook kan de HTML in de output cell weergeven.  
Het kan handig zijn om te zien wat de gekregen HTML representeerd.  
De HTML is zonder CSS opmaak dus kan er anders uit zien dan in de browser.  

In [11]:
# optioneel
from IPython import display
display.HTML(str(soup))

ACE,-,-,-
AN/FSQ-32,01 Jan 1960,-,IBM
AN/FSQ-7,01 Jan 1958,-,IBM
APEXC,-,-,-
ARRA,-,-,-


De data in de table van de HTML is nodig

In [12]:
table = soup.table
display.HTML(str(table))

ACE,-,-,-
AN/FSQ-32,01 Jan 1960,-,IBM
AN/FSQ-7,01 Jan 1958,-,IBM
APEXC,-,-,-
ARRA,-,-,-


Met een [list-comprehension] kan er  over de `<th>` tags geitereerd worden en zo uit elke tag de text filteren.  
Een list-comprehension is een loop in een data container zoals een `list`.  
De loop itereert en plaatst te objecten in een nieuwe data-container.  
list-comprehensions zijn efficient en flexibel.

[list-comprehension]: https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions

In [13]:
headers = [h.get_text() for h in table.thead.find_all('th')]
headers  # list met str objecten

['Computer name', 'Introduced', 'Discontinued', 'Company']

Nu er een lijst met headers is is er een indicatie welke cel wat bevat.  
De headers kan gebruikt worden als _key_ in een dict.  
De cellen in de regels zijn dan de _values_.  



In [44]:
rows = table.tbody.find_all('tr')
rows  # list met bs4.element.Tag objecten

[<tr><td><a href="/computers/381">ACE</a></td><td>-</td><td>-</td><td>-</td></tr>,
 <tr><td><a href="/computers/501">AN/FSQ-32</a></td><td>01 Jan 1960</td><td>-</td><td>IBM</td></tr>,
 <tr><td><a href="/computers/500">AN/FSQ-7</a></td><td>01 Jan 1958</td><td>-</td><td>IBM</td></tr>,
 <tr><td><a href="/computers/388">APEXC</a></td><td>-</td><td>-</td><td>-</td></tr>,
 <tr><td><a href="/computers/355">ARRA</a></td><td>-</td><td>-</td><td>-</td></tr>]

De headers en zijn de regels geparsed uit de table van de website.  
Nu kan deze data in een CSV file of JSON gezet worden.

Hieronder een voorbeeld hoe er een JSON van de data gemaakt kan worden.

In [55]:
import json

row_list = []

for row in rows:
    row_text = [r.get_text() for r in row]  # list comprehension om een list met text te maken
    # dict-zip
    header_row_dict = dict(zip(headers, row_text))
    row_list.append(header_row_dict)


# creeer de json
json.dumps({'rows': row_list})

'{"rows": [{"Computer name": "ACE", "Introduced": "-", "Discontinued": "-", "Company": "-"}, {"Computer name": "AN/FSQ-32", "Introduced": "01 Jan 1960", "Discontinued": "-", "Company": "IBM"}, {"Computer name": "AN/FSQ-7", "Introduced": "01 Jan 1958", "Discontinued": "-", "Company": "IBM"}, {"Computer name": "APEXC", "Introduced": "-", "Discontinued": "-", "Company": "-"}, {"Computer name": "ARRA", "Introduced": "-", "Discontinued": "-", "Company": "-"}]}'

Hieronder een voorbeeld hoe de er een CSV file met de data gemaakt kan worden.

In [54]:
import csv

filename = 'gatling_io.txt'

# `with` context manager
with open(filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    
    writer.writerow(headers)  # schrijf de header regel (fieldnames)
    for row in rows:
        row_text = [r.get_text() for r in row]
        writer.writerow(row_text)  # schrijf de regels


# open en lees de gemaakte file.
open(filename, 'r').read()

'Computer name,Introduced,Discontinued,Company\nACE,-,-,-\nAN/FSQ-32,01 Jan 1960,-,IBM\nAN/FSQ-7,01 Jan 1958,-,IBM\nAPEXC,-,-,-\nARRA,-,-,-\n'